# X-ray CT Geometry Tutorial

This notebook demonstrates how to use the `ct_geometry` function to calculate key parameters for X-ray computed tomography (CT) imaging, including:

- **Geometric magnification**
- **Voxel size**
- **Spatial resolution**
- **Sample projection on detector**

We'll also visualize the CT geometry setup with 2D schematic diagrams and explore how different parameters affect image quality.

## Important: SI Units Convention

All functions in `drp_template.compute` use **SI base units (meters)**. If your CT scanner specifications are in millimeters or micrometers, you must convert them first:

- **mm → m**: divide by 1000 (e.g., 26 mm = 0.026 m)
- **μm → m**: divide by 1,000,000 (e.g., 127 μm = 127e-6 m)

The `drp_template.compute` module provides convenience functions: `mm2m()`, `um2m()`, `m2mm()`, `m2um()` for easy conversion.

In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.patches import FancyBboxPatch, Circle, FancyArrowPatch
from drp_template.compute import ct_geometry, mm2m, um2m, m2mm, m2um

## 1. Understanding CT Geometry

In X-ray CT, the geometric setup consists of three main components aligned along an axis:

1. **X-ray Source** - Emits cone-beam X-rays from a focal spot
2. **Sample (Object)** - Placed between source and detector
3. **Detector** - Records the projected image

### Key Distances

- **SOD** (Source-to-Object Distance): Distance from X-ray source to the center of the sample
- **SDD** (Source-to-Detector Distance): Distance from X-ray source to the detector
- **ODD** (Object-to-Detector Distance): SDD - SOD

### Geometric Magnification

The magnification is determined by the ratio:

$$M = \frac{SDD}{SOD}$$

This magnification affects:
- How large the sample appears on the detector
- The effective voxel size in the reconstructed image
- The spatial resolution achievable

In [ ]:
# --- CT Geometry Plotting Utilities (Side vs Top Views) ---
import matplotlib.pyplot as plt
from matplotlib.patches import FancyBboxPatch, Circle

def _compute_base_geometry(SOD, SDD):
    """Compute distances and horizontal scaling shared by both views.
    Returns dict with ODD, magnification, scale, source_x, object_x, detector_x.
    Horizontal scale maps SDD span to 10 arbitrary units for visual consistency."""
    if SOD <= 0 or SDD <= 0 or SDD <= SOD:
        raise ValueError("Require SOD > 0, SDD > 0 and SDD > SOD")
    ODD = SDD - SOD
    magnification = SDD / SOD
    scale = 10.0 / SDD  # arbitrary normalization so that source->detector = 10 units
    source_x = 0.0
    object_x = SOD * scale
    detector_x = SDD * scale
    return dict(ODD=ODD, magnification=magnification, scale=scale,
                source_x=source_x, object_x=object_x, detector_x=detector_x)

# ----------------------------- SIDE VIEW (height-driven) -----------------------------
def plot_ct_side_view(ax, SOD, SDD, sample_height, detector_height, focal_spot_size=None, show_projections=True):
    """Plot side view of CT geometry.

    Parameters
    ----------
    ax : matplotlib Axes
        Target axes.
    SOD, SDD : float (m)
        Source-to-object and source-to-detector distances (SI units).
    sample_height : float (m)
        Physical height of the cylindrical sample (out-of-plane diameter not needed here).
    detector_height : float (m)
        Active height of the detector.
    focal_spot_size : float, optional (m)
        Focal spot size for scaled source marker.
    show_projections : bool
        If True, show magnified projection height at detector.

    Notes
    -----
    This view emphasizes vertical (height) dimension. The projected height is magnified by M = SDD/SOD.
    """
    g = _compute_base_geometry(SOD, SDD)
    ODD = g['ODD']
    M = g['magnification']
    scale = g['scale']
    source_x = g['source_x']
    object_x = g['object_x']
    detector_x = g['detector_x']
    projected_height = sample_height * M
    # Debug
    print(f"[DEBUG][SIDE] SOD: {source_x:.2f}->{object_x:.2f}  SDD: {source_x:.2f}->{detector_x:.2f}  ODD: {object_x:.2f}->{detector_x:.2f}")
    # Source (exaggerated)
    source_radius = (focal_spot_size * scale * 50) if focal_spot_size else 0.3
    ax.add_patch(Circle((source_x, 0), source_radius, color='red', zorder=10, label='Source'))
    # Sample (rectangle – narrow width just illustrative)
    sample_h_scaled = sample_height * scale
    sample_width_scaled = 0.3  # illustrative thickness
    ax.add_patch(FancyBboxPatch((object_x - sample_width_scaled/2, -sample_h_scaled/2),
                                sample_width_scaled, sample_h_scaled,
                                boxstyle='round,pad=0.05', facecolor='sandybrown', edgecolor='none',
                                zorder=5, label=f'Sample (h={sample_height:.2e} m)'))
    # Detector (height scaled)
    det_h_scaled = detector_height * scale
    ax.add_patch(plt.Rectangle((detector_x - 0.1, -det_h_scaled/2), 0.2, det_h_scaled,
                               facecolor='gray', edgecolor='black', linewidth=2, zorder=4,
                               label=f'Detector (H={detector_height:.2e} m)'))
    # Projection (height)
    if show_projections:
        proj_h_scaled = projected_height * scale
        for sign in (1, -1):
            ax.plot([source_x, object_x], [0, sign * (sample_h_scaled/2)], 'r--', alpha=0.6, linewidth=1)
            ax.plot([object_x, detector_x], [sign * (sample_h_scaled/2), sign * (proj_h_scaled/2)], 'r--', alpha=0.6, linewidth=1)
        ax.add_line(plt.Line2D([detector_x + 0.2, detector_x + 0.2],
                                [-proj_h_scaled/2, proj_h_scaled/2],
                                color='orange', linewidth=4, label=f'Projected h ({projected_height:.2e} m)'))
    # Distance annotations (horizontal)
    _annotate_distances(ax, source_x, object_x, detector_x, SOD, ODD, SDD, scale, reference_span=max(det_h_scaled, sample_h_scaled, (projected_height*scale if show_projections else sample_h_scaled)))
    _finalize_axes(ax, detector_x, title='CT Geometry – Side View')
    return ax

# ----------------------------- TOP VIEW (diameter-driven) -----------------------------
def plot_ct_top_view(ax, SOD, SDD, sample_diameter, detector_width, focal_spot_size=None, show_projections=True):
    """Plot top (plan) view of CT geometry without diameter annotation; projected size drawn as a circle behind detector.

    Parameters
    ----------
    SOD, SDD : float (m)
        Geometry distances.
    sample_diameter : float (m)
        Cylindrical sample diameter.
    detector_width : float (m)
        Active detector width drawn vertically for consistency with side view.
    focal_spot_size : float, optional (m)
        Focal spot size for source marker.
    show_projections : bool
        If True, draw red projection rays and a projected circle behind the detector.
    """
    g = _compute_base_geometry(SOD, SDD)
    ODD = g['ODD']
    M = g['magnification']
    scale = g['scale']
    source_x = g['source_x']
    object_x = g['object_x']
    detector_x = g['detector_x']
    projected_diameter = sample_diameter * M
    # Debug
    print(f"[DEBUG][TOP ] SOD: {source_x:.2f}->{object_x:.2f}  SDD: {source_x:.2f}->{detector_x:.2f}  ODD: {object_x:.2f}->{detector_x:.2f}")
    # Source
    source_radius = (focal_spot_size * scale * 50) if focal_spot_size else 0.3
    ax.add_patch(Circle((source_x, 0), source_radius, color='red', zorder=10, label='Source'))
    # Sample circle
    sample_radius_scaled = (sample_diameter * scale) / 2
    ax.add_patch(Circle((object_x, 0), sample_radius_scaled, facecolor='sandybrown', edgecolor='none', zorder=5,
                        label=f'Sample (Ø={sample_diameter:.2e} m)'))
    # Detector (width shown vertically)
    det_w_scaled = detector_width * scale
    ax.add_patch(plt.Rectangle((detector_x - 0.1, -det_w_scaled/2), 0.2, det_w_scaled,
                               facecolor='gray', edgecolor='black', linewidth=2, zorder=4,
                               label=f'Detector (W={detector_width:.2e} m)'))
    # Projections: red rays and projected circle behind detector
    if show_projections:
        proj_radius_scaled = (projected_diameter * scale) / 2
        # Rays (top and bottom)
        for sign in (1, -1):
            ax.plot([source_x, object_x], [0, sign * (sample_radius_scaled)], 'r--', alpha=0.6, linewidth=1)
            ax.plot([object_x, detector_x], [sign * (sample_radius_scaled), sign * (proj_radius_scaled)], 'r--', alpha=0.6, linewidth=1)
        # Projected circle behind detector
        circle_offset = 0.8
        proj_center_x = detector_x + circle_offset
        ax.add_patch(Circle((proj_center_x, 0), proj_radius_scaled, facecolor='none', edgecolor='orange', linewidth=3, zorder=4,
                            label=f'Projected circle (Ø={projected_diameter:.2e} m)'))
        # Extend x-limits logic if needed by passing a larger right bound
        _annotate_distances(ax, source_x, object_x, detector_x, SOD, ODD, SDD, scale, reference_span=max(det_w_scaled, sample_radius_scaled*2, projected_diameter*scale))
        _finalize_axes(ax, proj_center_x + proj_radius_scaled, title='CT Geometry – Top View')
        return ax
    # Without projections, annotate distances and finalize with default right bound
    _annotate_distances(ax, source_x, object_x, detector_x, SOD, ODD, SDD, scale, reference_span=max(det_w_scaled, sample_radius_scaled*2))
    _finalize_axes(ax, detector_x, title='CT Geometry – Top View')
    return ax

# ----------------------------- Shared helpers -----------------------------
def _annotate_distances(ax, source_x, object_x, detector_x, SOD, ODD, SDD, scale, reference_span):
    """Add SOD, ODD, SDD horizontal annotations above/below drawing.
    reference_span is a vertical size used to position annotations outside objects."""
    half_ref = reference_span / 2
    annotation_offset = 0.3
    annotation_spacing = 0.6
    text_offset = 0.3
    y1 = -(half_ref + annotation_offset)
    y2 = y1 - annotation_spacing
    y_top = half_ref + annotation_offset
    # SOD
    ax.annotate('', xy=(object_x, y1), xytext=(source_x, y1), arrowprops=dict(arrowstyle='<->', color='green', lw=2))
    ax.text((source_x + object_x)/2, y1 - text_offset, f'SOD = {SOD:.3e} m', ha='center', fontsize=11, color='green', fontweight='bold')
    # ODD
    ax.annotate('', xy=(detector_x, y2), xytext=(object_x, y2), arrowprops=dict(arrowstyle='<->', color='purple', lw=2))
    ax.text((object_x + detector_x)/2, y2 - text_offset, f'ODD = {ODD:.3e} m', ha='center', fontsize=11, color='purple', fontweight='bold')
    # SDD
    ax.annotate('', xy=(detector_x, y_top), xytext=(source_x, y_top), arrowprops=dict(arrowstyle='<->', color='darkblue', lw=2))
    ax.text((source_x + detector_x)/2, y_top + text_offset, f'SDD = {SDD:.3e} m', ha='center', fontsize=11, color='darkblue', fontweight='bold')
    # Legend placeholders (distance)
    ax.plot([], [], color='green', label='SOD')
    ax.plot([], [], color='purple', label='ODD')
    ax.plot([], [], color='darkblue', label='SDD')
    ax.plot([], [], color='orange', label='Projected size')

def _finalize_axes(ax, right_bound_x, title):
    xlim_left = -1
    xlim_right = right_bound_x + 2
    x_range = xlim_right - xlim_left
    y_range = x_range / (14.0 / 8.0)
    ax.set_xlim(xlim_left, xlim_right)
    ax.set_ylim(-y_range/2, y_range/2)
    ax.set_aspect('equal')
    ax.set_title(title, fontsize=14, fontweight='bold')
    ax.grid(True, alpha=0.2)
    ax.set_xticks([]); ax.set_yticks([])
    ax.legend(loc='upper left', fontsize=9, ncol=2)

# ----------------------------- Convenience wrappers -----------------------------
def plot_ct_side_view_only(SOD, SDD, sample_height, detector_height, focal_spot_size=None, show_projections=True, figsize=(14,6)):
    fig, ax = plt.subplots(figsize=figsize)
    plot_ct_side_view(ax, SOD, SDD, sample_height, detector_height, focal_spot_size=focal_spot_size, show_projections=show_projections)
    plt.tight_layout(); return fig, ax

def plot_ct_top_view_only(SOD, SDD, sample_diameter, detector_width, focal_spot_size=None, show_projections=True, figsize=(14,6)):
    fig, ax = plt.subplots(figsize=figsize)
    plot_ct_top_view(ax, SOD, SDD, sample_diameter, detector_width, focal_spot_size=focal_spot_size, show_projections=show_projections)
    plt.tight_layout(); return fig, ax

def plot_ct_geometry(SOD, SDD, sample_height, detector_height, sample_diameter, detector_width, focal_spot_size=None, show_projections=True, figsize=(14,12)):
    fig, axes = plt.subplots(2, 1, figsize=figsize)
    plot_ct_side_view(axes[0], SOD, SDD, sample_height, detector_height, focal_spot_size=focal_spot_size, show_projections=show_projections)
    plot_ct_top_view(axes[1], SOD, SDD, sample_diameter, detector_width, focal_spot_size=focal_spot_size, show_projections=show_projections)
    plt.tight_layout(); return fig, axes

## 2. Example: Typical Micro-CT Setup

Let's calculate the geometry for a typical micro-CT scanner with the following specifications:

- **SOD**: 26 mm (source-to-object distance)
- **SDD**: 127 mm (source-to-detector distance)
- **Detector pixel size**: 127 μm
- **Focal spot size**: 5 μm
- **Sample diameter**: 5 mm
- **Detector width**: 16.5 mm (for example, 130 pixels × 127 μm)

In [ ]:
# Define CT parameters (convert from mm/μm to meters)
SOD = mm2m(50)                  # 50 mm → meters
SDD = mm2m(127)                 # 127 mm → meters
detector_pixel_size = um2m(127) # 127 μm → meters
focal_spot_size = um2m(5)       # 5 μm → meters
sample_diameter = mm2m(40)      # 40 mm → meters
sample_height = mm2m(100)       # 100 mm → meters
detector_height = mm2m(320)     # 320 mm → meters
detector_width = mm2m(320)      # 320 mm → meters

# Display parameters
print("CT Scanner Parameters (in SI units - meters):")
print(f"  SOD: {SOD} m ({m2mm(SOD):.1f} mm)")
print(f"  SDD: {SDD} m ({m2mm(SDD):.1f} mm)")
print(f"  Detector pixel size: {detector_pixel_size} m ({m2um(detector_pixel_size):.1f} μm)")
print(f"  Focal spot size: {focal_spot_size} m ({m2um(focal_spot_size):.1f} μm)")
print(f"  Sample diameter: {sample_diameter} m ({m2mm(sample_diameter):.1f} mm)")
print(f"  Sample height: {sample_height} m ({m2mm(sample_height):.1f} mm)")
print(f"  Detector width: {detector_width} m ({m2mm(detector_width):.1f} mm)")
print(f"  Detector height: {detector_height} m ({m2mm(detector_height):.1f} mm)")

In [ ]:
# Calculate CT geometry
results = ct_geometry(
    SOD=SOD,
    SDD=SDD,
    detector_pixel_size=detector_pixel_size,
    focal_spot_size=focal_spot_size,
    sample_diameter=sample_diameter,
    detector_width=detector_width
)

print("\nCalculated Results:")
print(f"  Geometric magnification: {results['geometric_magnification']:.3f}")
print(f"  Voxel size: {m2um(results['voxel_size_m']):.2f} μm")
print(f"  Spatial resolution: {m2um(results['spatial_resolution_m']):.2f} μm")
print(f"  Projected sample diameter: {m2mm(results['projected_sample_diameter_m']):.2f} mm")
print(f"  Detector utilization: {results['projected_sample_diameter_m']/detector_width*100:.1f}%")

## Side, Top, and Combined View Examples
Below are standalone examples to draw each view individually and the combined (2×1) figure. These reuse the CT parameters defined above.

In [ ]:
# Example: side view only (height-driven)
_ = plt.close('all')
fig_side, ax_side = plot_ct_side_view_only(
    SOD, SDD, sample_height=sample_height, detector_height=detector_height,
    focal_spot_size=focal_spot_size,
    show_projections=True,
    figsize=(14, 6)
 )
plt.show()

In [ ]:
# Example: top view only (diameter-driven)
fig_top, ax_top = plot_ct_top_view_only(
    SOD, SDD, sample_diameter=sample_diameter, detector_width=detector_width,
    focal_spot_size=focal_spot_size,
    show_projections=True,
    figsize=(14, 6)
 )
plt.show()

In [ ]:
# Example: combined side (height) + top (diameter) views
fig_combo, axes_combo = plot_ct_geometry(
    SOD, SDD,
    sample_height=sample_height, detector_height=detector_height,
    sample_diameter=sample_diameter, detector_width=detector_width,
    focal_spot_size=focal_spot_size,
    show_projections=True,
    figsize=(14, 12)
 )
plt.show()

In [ ]:
# Visualize the CT geometry
plot_ct_geometry(SOD, SDD, sample_diameter, detector_width, 
                 focal_spot_size=focal_spot_size, show_projections=True)
plt.show()

## 3. Parameter Sensitivity Analysis

Let's explore how changing the source-to-object distance (SOD) affects the key parameters while keeping SDD constant.

In [ ]:
# Vary SOD from 10 mm to 100 mm
SOD_range_mm = np.linspace(10, 100, 50)
SOD_range = mm2m(SOD_range_mm)

# Arrays to store results
magnifications = []
voxel_sizes_um = []
resolutions_um = []
projected_diameters_mm = []

for sod in SOD_range:
    try:
        result = ct_geometry(
            SOD=sod,
            SDD=SDD,
            detector_pixel_size=detector_pixel_size,
            focal_spot_size=focal_spot_size,
            sample_diameter=sample_diameter,
            detector_width=detector_width
        )
        magnifications.append(result['geometric_magnification'])
        voxel_sizes_um.append(m2um(result['voxel_size_m']))
        resolutions_um.append(m2um(result['spatial_resolution_m']))
        projected_diameters_mm.append(m2mm(result['projected_sample_diameter_m']))
    except ValueError:
        # Sample projection exceeds detector
        magnifications.append(np.nan)
        voxel_sizes_um.append(np.nan)
        resolutions_um.append(np.nan)
        projected_diameters_mm.append(np.nan)

In [ ]:
# Create multi-panel plot
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Plot 1: Magnification vs SOD
axes[0, 0].plot(SOD_range_mm, magnifications, 'b-', linewidth=2)
axes[0, 0].axvline(m2mm(SOD), color='red', linestyle='--', label='Current SOD')
axes[0, 0].set_xlabel('SOD (mm)', fontsize=11)
axes[0, 0].set_ylabel('Geometric Magnification', fontsize=11)
axes[0, 0].set_title('Magnification vs. Source-to-Object Distance', fontsize=12, fontweight='bold')
axes[0, 0].grid(True, alpha=0.3)
axes[0, 0].legend()

# Plot 2: Voxel Size vs SOD
axes[0, 1].plot(SOD_range_mm, voxel_sizes_um, 'g-', linewidth=2)
axes[0, 1].axvline(m2mm(SOD), color='red', linestyle='--', label='Current SOD')
axes[0, 1].set_xlabel('SOD (mm)', fontsize=11)
axes[0, 1].set_ylabel('Voxel Size (μm)', fontsize=11)
axes[0, 1].set_title('Voxel Size vs. Source-to-Object Distance', fontsize=12, fontweight='bold')
axes[0, 1].grid(True, alpha=0.3)
axes[0, 1].legend()

# Plot 3: Spatial Resolution vs SOD
axes[1, 0].plot(SOD_range_mm, resolutions_um, 'orange', linewidth=2)
axes[1, 0].axvline(m2mm(SOD), color='red', linestyle='--', label='Current SOD')
axes[1, 0].set_xlabel('SOD (mm)', fontsize=11)
axes[1, 0].set_ylabel('Spatial Resolution (μm)', fontsize=11)
axes[1, 0].set_title('Spatial Resolution vs. Source-to-Object Distance', fontsize=12, fontweight='bold')
axes[1, 0].grid(True, alpha=0.3)
axes[1, 0].legend()

# Plot 4: Projected Sample Diameter vs SOD
axes[1, 1].plot(SOD_range_mm, projected_diameters_mm, 'purple', linewidth=2, label='Projected sample')
axes[1, 1].axhline(m2mm(detector_width), color='black', linestyle='--', linewidth=2, label='Detector width')
axes[1, 1].axvline(m2mm(SOD), color='red', linestyle='--', label='Current SOD')
axes[1, 1].set_xlabel('SOD (mm)', fontsize=11)
axes[1, 1].set_ylabel('Diameter (mm)', fontsize=11)
axes[1, 1].set_title('Projected Sample Diameter vs. SOD', fontsize=12, fontweight='bold')
axes[1, 1].grid(True, alpha=0.3)
axes[1, 1].legend()
axes[1, 1].fill_between(SOD_range_mm, 0, m2mm(detector_width), 
                         alpha=0.2, color='green', label='Valid range')

plt.tight_layout()
plt.show()

### Key Observations:

1. **Magnification** decreases as SOD increases (moving sample closer to detector)
2. **Voxel size** increases with SOD (less magnification = larger voxels)
3. **Spatial resolution** also degrades with larger SOD
4. **Projected sample size** decreases with SOD - must stay within detector width

**Trade-off**: Smaller SOD gives better resolution but higher magnification, which can cause the sample projection to exceed the detector width.

## 4. Comparing Different CT Configurations

Let's compare three different geometric configurations side by side:

In [ ]:
# Define three configurations
configs = {
    'High Resolution': {
        'SOD': mm2m(20),
        'SDD': mm2m(60),
        'sample_diameter': mm2m(3),
    },
    'Standard': {
        'SOD': mm2m(26),
        'SDD': mm2m(60),
        'sample_diameter': mm2m(5),
    },
    'Large Sample': {
        'SOD': mm2m(50),
        'SDD': mm2m(60),
        'sample_diameter': mm2m(10),
    }
}

# Calculate and display results
print("Configuration Comparison:")
print("=" * 90)
print(f"{'Config':<15} {'Ø (mm)':<8} {'SOD (mm)':<12} {'Geom. Magnification':<20} {'Resolution (μm)':<18} {'Projected Ø (mm)':<12}")
print("=" * 90)

for name, params in configs.items():
    result = ct_geometry(
        SOD=params['SOD'],
        SDD=params['SDD'],
        detector_pixel_size=detector_pixel_size,
        focal_spot_size=focal_spot_size,
        sample_diameter=params['sample_diameter'],
        detector_width=detector_width
    )
    
    print(f"{name:<15} {m2mm(params['sample_diameter']):<8.1f} {m2mm(params['SOD']):<12.1f} {result['geometric_magnification']:<8.2f} "
          f"{m2um(result['spatial_resolution_m']):<18.2f} "
          f"{m2mm(result['projected_sample_diameter_m']):<12.2f}")

print("=" * 90)

In [ ]:
# Visualize all three configurations
for idx, (name, params) in enumerate(configs.items()):
    fig, axes = plot_ct_geometry(
        SOD=params['SOD'],
        SDD=params['SDD'],
        sample_diameter=params['sample_diameter'],
        detector_width=detector_width,
        focal_spot_size=focal_spot_size,
        show_projections=True
    )
    fig.suptitle(f'{name} Configuration', fontsize=13, fontweight='bold')
    plt.tight_layout()
    plt.show()

## 5. Interactive Exploration

Try your own CT parameters! Adjust the values below to see how they affect the geometry.

In [ ]:
# === CUSTOMIZE YOUR CT PARAMETERS HERE ===

# Distances (enter in mm, will be converted to meters)
my_SOD_mm = 30.0
my_SDD_mm = 150.0

# Detector specifications (pixel size in μm, width in mm)
my_detector_pixel_um = 100.0
my_detector_width_mm = 20.0

# X-ray source focal spot (in μm)
my_focal_spot_um = 8.0

# Sample diameter (in mm)
my_sample_diameter_mm = 6.0

# Convert to SI units
my_SOD = mm2m(my_SOD_mm)
my_SDD = mm2m(my_SDD_mm)
my_detector_pixel = um2m(my_detector_pixel_um)
my_detector_width = mm2m(my_detector_width_mm)
my_focal_spot = um2m(my_focal_spot_um)
my_sample_diameter = mm2m(my_sample_diameter_mm)

# Calculate geometry
try:
    my_results = ct_geometry(
        SOD=my_SOD,
        SDD=my_SDD,
        detector_pixel_size=my_detector_pixel,
        focal_spot_size=my_focal_spot,
        sample_diameter=my_sample_diameter,
        detector_width=my_detector_width
    )
    
    print("✓ Valid Configuration!")
    print("\nYour CT Setup:")
    print(f"  Geometric Magnification: {my_results['geometric_magnification']:.3f}×")
    print(f"  Voxel Size: {m2um(my_results['voxel_size_m']):.2f} μm")
    print(f"  Spatial Resolution: {m2um(my_results['spatial_resolution_m']):.2f} μm")
    print(f"  Projected Sample: {m2mm(my_results['projected_sample_diameter_m']):.2f} mm")
    print(f"  Detector Utilization: {my_results['projected_sample_diameter_m']/my_detector_width*100:.1f}%")
    
    # Visualize
    plot_ct_geometry(my_SOD, my_SDD, my_sample_diameter, my_detector_width,
                     focal_spot_size=my_focal_spot, show_projections=True)
    plt.show()
    
except ValueError as e:
    print(f"✗ Configuration Error: {e}")
    print("\nTry:")
    print("  - Increasing SOD (move sample away from source)")
    print("  - Decreasing sample diameter")
    print("  - Using a wider detector")

## 6. Resolution Limits: Voxel Size vs. Focal Spot

The spatial resolution in CT is limited by two factors:

1. **Voxel size**: Determined by detector pixel size and magnification
2. **Focal spot blurring**: Geometric unsharpness from finite source size

The effective resolution is the **maximum** of these two values. Let's visualize this relationship:

In [ ]:
# Calculate voxel size and focal spot contribution separately
voxel_contribution = []
focal_spot_contribution = []
total_resolution = []

for sod in SOD_range:
    mag = SDD / sod
    voxel_size = detector_pixel_size / mag
    effective_focal = focal_spot_size / mag
    
    voxel_contribution.append(m2um(voxel_size))
    focal_spot_contribution.append(m2um(effective_focal))
    total_resolution.append(m2um(max(voxel_size, effective_focal)))

# Plot
fig, ax = plt.subplots(figsize=(12, 7))

ax.plot(SOD_range_mm, voxel_contribution, 'b-', linewidth=2, label='Voxel size limit')
ax.plot(SOD_range_mm, focal_spot_contribution, 'r--', linewidth=2, label='Focal spot blur')
ax.plot(SOD_range_mm, total_resolution, 'g-', linewidth=3, label='Effective resolution (max)')
ax.axvline(m2mm(SOD), color='orange', linestyle=':', linewidth=2, label='Current SOD')

ax.fill_between(SOD_range_mm, voxel_contribution, alpha=0.2, color='blue', label='Voxel-limited region')
ax.fill_between(SOD_range_mm, focal_spot_contribution, alpha=0.2, color='red', label='Blur-limited region')

ax.set_xlabel('Source-to-Object Distance (mm)', fontsize=12)
ax.set_ylabel('Resolution (μm)', fontsize=12)
ax.set_title('Resolution Contributions: Voxel Size vs. Focal Spot Blur', fontsize=13, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nInterpretation:")
print("- At small SOD (high magnification): Resolution limited by voxel size")
print("- At large SOD (low magnification): Resolution limited by focal spot blur")
print("- The effective resolution is always the worse of the two")

## 7. Summary and Best Practices

### Key Formulas

- **Geometric Magnification**: $M = \frac{SDD}{SOD}$
- **Voxel Size**: $v = \frac{d_{pixel}}{M}$ where $d_{pixel}$ is detector pixel size
- **Effective Focal Spot**: $f_{eff} = \frac{f_{source}}{M}$ where $f_{source}$ is focal spot size
- **Spatial Resolution**: $r = \max(v, f_{eff})$

### Design Guidelines

1. **For maximum resolution**:
   - Use smallest possible SOD (but check sample projection fits detector)
   - Use smallest focal spot size available
   - Use detector with smallest pixel size

2. **For large samples**:
   - Increase SOD to reduce magnification
   - Accept reduced resolution
   - Ensure projected sample < detector width

3. **Detector utilization**:
   - Aim for 70-90% detector utilization for optimal field of view
   - Too small: Wasting detector pixels
   - Too large: Risk of projection exceeding detector

### Function Reference

```python
from drp_template.compute import ct_geometry, mm2m, um2m

results = ct_geometry(
    SOD=mm2m(26),              # Convert mm to meters
    SDD=mm2m(127),
    detector_pixel_size=um2m(127),  # Convert μm to meters
    focal_spot_size=um2m(5),
    sample_diameter=mm2m(5),
    detector_width=mm2m(16.5)
)
```

**Returns**: Dictionary with `geometric_magnification`, `voxel_size_m`, `spatial_resolution_m`, `projected_sample_diameter_m`